In [180]:
#fpip install catboost

In [181]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier

In [182]:
#Não há nessidade de pre-processamento com o catboost, basta informar quais são as variáveis categóricas

df = pd.read_csv(f'{os.getcwd()}\\Tratamento e Pre-Processamento\\Dados\\heart_tratado.csv', sep=';')

previsores = df.iloc[:, 0:11]
alvo = df.iloc[:,11]

In [209]:
previsores_treino, previsores_teste, alvo_treino, alvo_teste = train_test_split(previsores, alvo, test_size = 0.3, 
                                                                                 random_state = 0)


In [184]:
variaveis_categoricas = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [185]:
catboost = CatBoostClassifier(task_type = 'CPU', iterations=100, learning_rate = 0.1, depth = 5, random_state = 5, 
                              eval_metric='Accuracy')

In [186]:
catboost.fit(previsores_treino, alvo_treino, cat_features = variaveis_categoricas, plot = True, 
             eval_set = (previsores_teste, alvo_teste))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7815913	test: 0.7753623	best: 0.7753623 (0)	total: 19.1ms	remaining: 1.89s
1:	learn: 0.8346334	test: 0.8115942	best: 0.8115942 (1)	total: 30.8ms	remaining: 1.51s
2:	learn: 0.8439938	test: 0.8007246	best: 0.8115942 (1)	total: 36.4ms	remaining: 1.18s
3:	learn: 0.8439938	test: 0.8007246	best: 0.8115942 (1)	total: 49.3ms	remaining: 1.18s
4:	learn: 0.8580343	test: 0.8188406	best: 0.8188406 (4)	total: 64.9ms	remaining: 1.23s
5:	learn: 0.8658346	test: 0.8260870	best: 0.8260870 (5)	total: 79.4ms	remaining: 1.24s
6:	learn: 0.8564743	test: 0.8152174	best: 0.8260870 (5)	total: 91.9ms	remaining: 1.22s
7:	learn: 0.8486739	test: 0.8079710	best: 0.8260870 (5)	total: 106ms	remaining: 1.22s
8:	learn: 0.8455538	test: 0.8079710	best: 0.8260870 (5)	total: 119ms	remaining: 1.21s
9:	learn: 0.8439938	test: 0.8043478	best: 0.8260870 (5)	total: 133ms	remaining: 1.2s
10:	learn: 0.8439938	test: 0.8007246	best: 0.8260870 (5)	total: 149ms	remaining: 1.21s
11:	learn: 0.8439938	test: 0.8043478	best: 0.82

96:	learn: 0.8985959	test: 0.8659420	best: 0.8695652 (49)	total: 1.42s	remaining: 43.9ms
97:	learn: 0.8985959	test: 0.8659420	best: 0.8695652 (49)	total: 1.43s	remaining: 29.3ms
98:	learn: 0.8985959	test: 0.8659420	best: 0.8695652 (49)	total: 1.45s	remaining: 14.6ms
99:	learn: 0.8985959	test: 0.8659420	best: 0.8695652 (49)	total: 1.46s	remaining: 0us

bestTest = 0.8695652174
bestIteration = 49

Shrink model to first 50 iterations.


In [187]:
previsoes_cat = catboost.predict(previsores_teste)

In [188]:
print('Acurácia: %.2f%%' % (accuracy_score(alvo_teste, previsoes_cat) * 100))

Acurácia: 86.96%


In [189]:
confusion_matrix(alvo_teste, previsoes_cat)

array([[103,  18],
       [ 18, 137]], dtype=int64)

In [190]:
print(classification_report(alvo_teste, previsoes_cat))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       121
           1       0.88      0.88      0.88       155

    accuracy                           0.87       276
   macro avg       0.87      0.87      0.87       276
weighted avg       0.87      0.87      0.87       276



In [191]:
previsoes_treino = catboost.predict(previsores_treino)

In [192]:
accuracy_score(alvo_treino, previsoes_treino)

0.8939157566302652

In [193]:
confusion_matrix(alvo_treino, previsoes_treino)

array([[247,  42],
       [ 26, 326]], dtype=int64)

In [194]:
#Validação Cruzada

In [198]:
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [204]:
previsores_file = open(f'{os.getcwd()}\\Tratamento e Pre-Processamento\\Dados\\heart_previsores.pkl', 'rb')
previsores_pp = pickle.load(previsores_file)
previsores_file.close()


alvo_file = open(f'{os.getcwd()}\\Tratamento e Pre-Processamento\\Dados\\heart_alvo.pkl', 'rb')
alvo_pp = pickle.load(alvo_file)
alvo_file.close()

In [205]:
modelo =  CatBoostClassifier(task_type = 'CPU', iterations=100, learning_rate = 0.1, depth = 5, random_state = 5, 
                              eval_metric='Accuracy')
resultado = cross_val_score(modelo, previsores_pp, alvo_pp, cv = kfold)

0:	learn: 0.8510158	total: 1.29ms	remaining: 128ms
1:	learn: 0.8577878	total: 2.22ms	remaining: 109ms
2:	learn: 0.8668172	total: 3.18ms	remaining: 103ms
3:	learn: 0.8679458	total: 4.09ms	remaining: 98.2ms
4:	learn: 0.8668172	total: 5.09ms	remaining: 96.7ms
5:	learn: 0.8690745	total: 6.05ms	remaining: 94.9ms
6:	learn: 0.8702032	total: 6.96ms	remaining: 92.5ms
7:	learn: 0.8702032	total: 7.93ms	remaining: 91.2ms
8:	learn: 0.8656885	total: 8.85ms	remaining: 89.4ms
9:	learn: 0.8656885	total: 9.79ms	remaining: 88.1ms
10:	learn: 0.8713318	total: 10.8ms	remaining: 87.1ms
11:	learn: 0.8735892	total: 11.7ms	remaining: 86ms
12:	learn: 0.8713318	total: 12.6ms	remaining: 84.5ms
13:	learn: 0.8758465	total: 13.6ms	remaining: 83.6ms
14:	learn: 0.8803612	total: 14.7ms	remaining: 83.1ms
15:	learn: 0.8747178	total: 15.6ms	remaining: 81.9ms
16:	learn: 0.8769752	total: 16.5ms	remaining: 80.7ms
17:	learn: 0.8781038	total: 17.6ms	remaining: 80ms
18:	learn: 0.8735892	total: 18.6ms	remaining: 79.3ms
19:	learn:

57:	learn: 0.9006772	total: 57.3ms	remaining: 41.5ms
58:	learn: 0.9006772	total: 58.5ms	remaining: 40.6ms
59:	learn: 0.9006772	total: 59.7ms	remaining: 39.8ms
60:	learn: 0.9018059	total: 60.7ms	remaining: 38.8ms
61:	learn: 0.9040632	total: 61.6ms	remaining: 37.8ms
62:	learn: 0.9040632	total: 62.6ms	remaining: 36.8ms
63:	learn: 0.9040632	total: 63.8ms	remaining: 35.9ms
64:	learn: 0.9040632	total: 64.7ms	remaining: 34.8ms
65:	learn: 0.9040632	total: 65.6ms	remaining: 33.8ms
66:	learn: 0.9051919	total: 66.6ms	remaining: 32.8ms
67:	learn: 0.9051919	total: 67.6ms	remaining: 31.8ms
68:	learn: 0.9051919	total: 68.7ms	remaining: 30.8ms
69:	learn: 0.9051919	total: 69.6ms	remaining: 29.8ms
70:	learn: 0.9051919	total: 70.6ms	remaining: 28.9ms
71:	learn: 0.9051919	total: 71.2ms	remaining: 27.7ms
72:	learn: 0.9051919	total: 72.2ms	remaining: 26.7ms
73:	learn: 0.9051919	total: 73.1ms	remaining: 25.7ms
74:	learn: 0.9051919	total: 74ms	remaining: 24.7ms
75:	learn: 0.9040632	total: 75ms	remaining: 23.7

16:	learn: 0.8735892	total: 16.5ms	remaining: 80.7ms
17:	learn: 0.8758465	total: 17.7ms	remaining: 80.5ms
18:	learn: 0.8781038	total: 18.7ms	remaining: 79.6ms
19:	learn: 0.8781038	total: 19.8ms	remaining: 79ms
20:	learn: 0.8792325	total: 20.5ms	remaining: 77.1ms
21:	learn: 0.8792325	total: 21.6ms	remaining: 76.5ms
22:	learn: 0.8826185	total: 22.7ms	remaining: 75.9ms
23:	learn: 0.8803612	total: 23.9ms	remaining: 75.5ms
24:	learn: 0.8837472	total: 24.8ms	remaining: 74.5ms
25:	learn: 0.8837472	total: 26.3ms	remaining: 74.9ms
26:	learn: 0.8882619	total: 27.5ms	remaining: 74.4ms
27:	learn: 0.8893905	total: 28.6ms	remaining: 73.4ms
28:	learn: 0.8882619	total: 29.7ms	remaining: 72.7ms
29:	learn: 0.8905192	total: 30.8ms	remaining: 71.9ms
30:	learn: 0.8905192	total: 31.9ms	remaining: 70.9ms
31:	learn: 0.8916479	total: 32.9ms	remaining: 69.9ms
32:	learn: 0.8905192	total: 34.1ms	remaining: 69.2ms
33:	learn: 0.8916479	total: 35.3ms	remaining: 68.5ms
34:	learn: 0.8916479	total: 36.3ms	remaining: 67

96:	learn: 0.9119639	total: 92ms	remaining: 2.85ms
97:	learn: 0.9119639	total: 93.3ms	remaining: 1.9ms
98:	learn: 0.9119639	total: 94.3ms	remaining: 952us
99:	learn: 0.9119639	total: 95.2ms	remaining: 0us
0:	learn: 0.8476298	total: 1.09ms	remaining: 108ms
1:	learn: 0.8510158	total: 2.06ms	remaining: 101ms
2:	learn: 0.8600451	total: 3.02ms	remaining: 97.5ms
3:	learn: 0.8555305	total: 3.92ms	remaining: 94.1ms
4:	learn: 0.8555305	total: 4.83ms	remaining: 91.9ms
5:	learn: 0.8600451	total: 5.78ms	remaining: 90.5ms
6:	learn: 0.8645598	total: 6.71ms	remaining: 89.2ms
7:	learn: 0.8656885	total: 7.63ms	remaining: 87.8ms
8:	learn: 0.8656885	total: 8.54ms	remaining: 86.4ms
9:	learn: 0.8679458	total: 9.5ms	remaining: 85.5ms
10:	learn: 0.8690745	total: 10.4ms	remaining: 84.3ms
11:	learn: 0.8724605	total: 11.6ms	remaining: 85ms
12:	learn: 0.8656885	total: 12.8ms	remaining: 85.3ms
13:	learn: 0.8713318	total: 14ms	remaining: 86ms
14:	learn: 0.8758465	total: 15ms	remaining: 85ms
15:	learn: 0.8690745	to

93:	learn: 0.9119639	total: 90.4ms	remaining: 5.77ms
94:	learn: 0.9130926	total: 91.5ms	remaining: 4.82ms
95:	learn: 0.9130926	total: 92.7ms	remaining: 3.86ms
96:	learn: 0.9130926	total: 94ms	remaining: 2.91ms
97:	learn: 0.9142212	total: 95.1ms	remaining: 1.94ms
98:	learn: 0.9142212	total: 96.2ms	remaining: 971us
99:	learn: 0.9142212	total: 97.3ms	remaining: 0us
0:	learn: 0.8397291	total: 1.23ms	remaining: 122ms
1:	learn: 0.8442438	total: 2.26ms	remaining: 111ms
2:	learn: 0.8634312	total: 3.23ms	remaining: 105ms
3:	learn: 0.8645598	total: 4.33ms	remaining: 104ms
4:	learn: 0.8690745	total: 5.35ms	remaining: 102ms
5:	learn: 0.8702032	total: 6.48ms	remaining: 101ms
6:	learn: 0.8747178	total: 7.59ms	remaining: 101ms
7:	learn: 0.8724605	total: 8.92ms	remaining: 103ms
8:	learn: 0.8645598	total: 10.1ms	remaining: 102ms
9:	learn: 0.8679458	total: 11.2ms	remaining: 101ms
10:	learn: 0.8724605	total: 12.4ms	remaining: 101ms
11:	learn: 0.8758465	total: 13.6ms	remaining: 99.7ms
12:	learn: 0.8724605

...

46:	learn: 0.9019166	total: 44.6ms	remaining: 50.3ms
47:	learn: 0.9007892	total: 45.6ms	remaining: 49.4ms
48:	learn: 0.9019166	total: 46.6ms	remaining: 48.5ms
49:	learn: 0.9041714	total: 47.8ms	remaining: 47.8ms
50:	learn: 0.9052988	total: 49ms	remaining: 47.1ms
51:	learn: 0.9064262	total: 50ms	remaining: 46.1ms
52:	learn: 0.9064262	total: 50.9ms	remaining: 45.2ms
53:	learn: 0.9064262	total: 51.9ms	remaining: 44.2ms
54:	learn: 0.9075536	total: 53.1ms	remaining: 43.5ms
55:	learn: 0.9098083	total: 54.1ms	remaining: 42.5ms
56:	learn: 0.9109357	total: 55.1ms	remaining: 41.5ms
57:	learn: 0.9109357	total: 55.9ms	remaining: 40.5ms
58:	learn: 0.9109357	total: 56.9ms	remaining: 39.5ms
59:	learn: 0.9109357	total: 57.9ms	remaining: 38.6ms
60:	learn: 0.9109357	total: 58.8ms	remaining: 37.6ms
61:	learn: 0.9109357	total: 59.7ms	remaining: 36.6ms
62:	learn: 0.9120631	total: 60.7ms	remaining: 35.7ms
63:	learn: 0.9131905	total: 61.7ms	remaining: 34.7ms
64:	learn: 0.9120631	total: 62.6ms	remaining: 33.7

44:	learn: 0.9041714	total: 42.4ms	remaining: 51.9ms
45:	learn: 0.9052988	total: 43.6ms	remaining: 51.2ms
46:	learn: 0.9052988	total: 44.7ms	remaining: 50.4ms
47:	learn: 0.9041714	total: 45.7ms	remaining: 49.5ms
48:	learn: 0.9041714	total: 46.6ms	remaining: 48.5ms
49:	learn: 0.9052988	total: 47.8ms	remaining: 47.8ms
50:	learn: 0.9064262	total: 48.7ms	remaining: 46.8ms
51:	learn: 0.9052988	total: 49.6ms	remaining: 45.8ms
52:	learn: 0.9052988	total: 50.6ms	remaining: 44.8ms
53:	learn: 0.9052988	total: 51.5ms	remaining: 43.8ms
54:	learn: 0.9064262	total: 52.5ms	remaining: 42.9ms
55:	learn: 0.9075536	total: 53.4ms	remaining: 42ms
56:	learn: 0.9086809	total: 54.4ms	remaining: 41ms
57:	learn: 0.9086809	total: 55.3ms	remaining: 40ms
58:	learn: 0.9086809	total: 56.2ms	remaining: 39ms
59:	learn: 0.9075536	total: 57.1ms	remaining: 38.1ms
60:	learn: 0.9075536	total: 58ms	remaining: 37.1ms
61:	learn: 0.9086809	total: 59ms	remaining: 36.1ms
62:	learn: 0.9120631	total: 59.9ms	remaining: 35.2ms
63:	l

46:	learn: 0.8917700	total: 44ms	remaining: 49.7ms
47:	learn: 0.8906426	total: 45.2ms	remaining: 48.9ms
48:	learn: 0.8928974	total: 46.1ms	remaining: 48ms
49:	learn: 0.8928974	total: 47ms	remaining: 47ms
50:	learn: 0.8917700	total: 47.9ms	remaining: 46ms
51:	learn: 0.8895152	total: 49ms	remaining: 45.2ms
52:	learn: 0.8928974	total: 49.9ms	remaining: 44.3ms
53:	learn: 0.8928974	total: 50.9ms	remaining: 43.3ms
54:	learn: 0.8951522	total: 51.8ms	remaining: 42.4ms
55:	learn: 0.8974070	total: 52.7ms	remaining: 41.4ms
56:	learn: 0.8951522	total: 53.7ms	remaining: 40.5ms
57:	learn: 0.8974070	total: 54.6ms	remaining: 39.5ms
58:	learn: 0.8974070	total: 55.5ms	remaining: 38.6ms
59:	learn: 0.8962796	total: 56.5ms	remaining: 37.7ms
60:	learn: 0.8985344	total: 57.5ms	remaining: 36.8ms
61:	learn: 0.8996618	total: 58.4ms	remaining: 35.8ms
62:	learn: 0.8996618	total: 59.4ms	remaining: 34.9ms
63:	learn: 0.9041714	total: 60.4ms	remaining: 33.9ms
64:	learn: 0.9041714	total: 61.3ms	remaining: 33ms
65:	lea

49:	learn: 0.9019166	total: 48.7ms	remaining: 48.7ms
50:	learn: 0.9019166	total: 49.9ms	remaining: 47.9ms
51:	learn: 0.9019166	total: 51.1ms	remaining: 47.2ms
52:	learn: 0.9019166	total: 52ms	remaining: 46.1ms
53:	learn: 0.9019166	total: 52.5ms	remaining: 44.7ms
54:	learn: 0.9041714	total: 53.4ms	remaining: 43.7ms
55:	learn: 0.9030440	total: 54.3ms	remaining: 42.7ms
56:	learn: 0.9052988	total: 55.3ms	remaining: 41.7ms
57:	learn: 0.9041714	total: 56.2ms	remaining: 40.7ms
58:	learn: 0.9052988	total: 57.1ms	remaining: 39.7ms
59:	learn: 0.9052988	total: 58ms	remaining: 38.7ms
60:	learn: 0.9052988	total: 59ms	remaining: 37.7ms
61:	learn: 0.9052988	total: 59.9ms	remaining: 36.7ms
62:	learn: 0.9052988	total: 60.9ms	remaining: 35.8ms
63:	learn: 0.9052988	total: 61.8ms	remaining: 34.8ms
64:	learn: 0.9041714	total: 62.8ms	remaining: 33.8ms
65:	learn: 0.9041714	total: 63.8ms	remaining: 32.9ms
66:	learn: 0.9041714	total: 64.7ms	remaining: 31.9ms
67:	learn: 0.9041714	total: 65.6ms	remaining: 30.9ms

46:	learn: 0.9019166	total: 47.6ms	remaining: 53.7ms
47:	learn: 0.9019166	total: 49ms	remaining: 53.1ms
48:	learn: 0.9030440	total: 50.2ms	remaining: 52.3ms
49:	learn: 0.9019166	total: 51.4ms	remaining: 51.4ms
50:	learn: 0.9007892	total: 52.6ms	remaining: 50.5ms
51:	learn: 0.9041714	total: 53.5ms	remaining: 49.4ms
52:	learn: 0.9041714	total: 54.4ms	remaining: 48.2ms
53:	learn: 0.9064262	total: 55.4ms	remaining: 47.2ms
54:	learn: 0.9075536	total: 56.5ms	remaining: 46.2ms
55:	learn: 0.9075536	total: 57.4ms	remaining: 45.1ms
56:	learn: 0.9075536	total: 58.3ms	remaining: 44ms
57:	learn: 0.9041714	total: 59.3ms	remaining: 42.9ms
58:	learn: 0.9064262	total: 60.2ms	remaining: 41.8ms
59:	learn: 0.9075536	total: 61.3ms	remaining: 40.9ms
60:	learn: 0.9098083	total: 62.5ms	remaining: 39.9ms
61:	learn: 0.9098083	total: 63.4ms	remaining: 38.9ms
62:	learn: 0.9098083	total: 64.4ms	remaining: 37.8ms
63:	learn: 0.9098083	total: 65.3ms	remaining: 36.7ms
64:	learn: 0.9086809	total: 66.3ms	remaining: 35.7

In [206]:
print('Acurácia Média: %.2f%%' % (resultado.mean() * 100))

Acurácia Média: 88.66%
